lean version of read in file for the NMX work flow (from simulatet event data to binned event data)


first add neded libaries

In [1]:
from datetime import datetime
startTime = datetime.now()
import tracemalloc
    #from scippnexus import data
#import scippnexus as snx
import numpy as np
    #qimport mantid_args
#from scippnexus import NXdetector
from scippneutron.conversion import graph

import scipp as sc
import scippneutron as scn
import scippnexus as snx

#%matplotlib widget


import plopp as pp
pp.patch_scipp()
import matplotlib.pyplot as plt
from plopp.widgets import Box
import ipywidgets as ipw
from matplotlib.colors import LogNorm

    #import multiprocessing as mp

import h5py
h5py.enable_ipython_completer()
%matplotlib inline
%matplotlib widget

# starting the momory monitoring
tracemalloc.start()


define general parameter

In [111]:
#Number op pixels per detecotr dimmension
pix = 1280
#number of time bins 
t_step = 1
#number of detectors
n_det = 3




pretreatment to make it faster
h5repack -l CHUNK=1024x6 2e11.h5 2e11-rechunk.h5 
or 
h5repack -l CHUNK=NONE 2e11.h5 2e11-nochunk.h5

h5repack -l CHUNK=1024x6 mccode.h5 mccode-nochunk.h5

In [107]:
fname = 14
if fname == 1:
   filename = '/Users/justinbergmann/work_flow/test_data/2e11.h5'
elif fname == 2:
    filename = '/Users/justinbergmann/work_flow/test_data/x0_3d.h5'
elif fname == 3:
    filename = '/Users/justinbergmann/work_flow/test_data/xe_0_NMX.h5'
elif fname == 4:
    filename = '/Users/justinbergmann/work_flow/test_data/xe_20_NMX.h5'
elif fname == 5:
    filename = '/Users/justinbergmann/work_flow/test_data/flip_det.h5'
elif fname == 6:
    filename = '/Users/justinbergmann/work_flow/test_data/flip_4.h5'
elif fname == 7:
    filename = '/Users/justinbergmann/work_flow/test_data/ye_0_NMX.h5'
elif fname == 8:
    filename = '/Users/justinbergmann/work_flow/test_data/xe1_12_NMX/xe1_12_no_chunk.h5'
elif fname == 9:
    filename = '/Users/justinbergmann/work_flow/test_data/1d.h5'

elif fname == 10:
    filename = '/Users/justinbergmann/work_flow/test_data/xflat20.h5'

elif fname == 11:
    filename = '/Users/justinbergmann/work_flow/test_data/small_vis/flat.nxs'

elif fname == 12:
    filename = '/Users/justinbergmann/work_flow/test_data/small_vis/shift_down.nxs'


elif fname == 13:
    filename = '/Users/justinbergmann/work_flow/test_data/mc2_7_2_test.h5'

elif fname == 14:
    filename = '/Users/justinbergmann/work_flow/test_data/mc2_new_comp.h5'



else:
    filename = '/Users/justinbergmann/work_flow/test_data/e12_no_chunk.h5'


f_vis = '/Users/justinbergmann/work_flow/test_data/ye_0_NMX_vis.nxs'

#f=snx.File(filename)    

read in event data and shape event list  give number of events

3D visulisation with file converted by mantid

In [ ]:
vis = sc.io.open_hdf5(filename = f_vis)
scn.instrument_view(vis.sum('tof'))

In [108]:
%%time
f = h5py.File(filename)
a = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['events'][...]
#a[0]
d = np.matrix.transpose(a)
print("shape of event list (p_x_y_n_id_t)", d.shape)
#alocate units to events and create seperate list for each parameter
x_list = sc.array(dims=['x'], unit='m', values=d[1])
y_list = sc.array(dims=['x'], unit='m', values=d[2])
t_list = sc.array(dims=['x'], unit='s', values=d[5])
id_list = sc.array(dims=['x'], unit=None, values=d[4], dtype='int64')
#print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)

#get evetns to seperated lists (x,y,t,id)
#x_list.unit = 'm'
#y_list.unit = 'm'
#t_list.unit = 'ms'
weights = sc.ones_like(x_list)
weights.unit = 'counts'
da = sc.DataArray(data=weights, coords={'x': x_list, 'y': y_list, 't': t_list, 'id': id_list})

#make sure alll IDs are reconised:
print("id min",id_list.values.min())
print("id max",id_list.values.max())

ids1 = sc.arange('id', 1, 1638401, unit=None)
ids2 = sc.arange('id', 2000001, 3638401, unit=None)
ids3 = sc.arange('id', 4000001, 5638401, unit=None)
ids = sc.concat([ids1, ids2, ids3], 'id')
#grouping by IDs
da3 = da.group(ids).fold('id', sizes={'panel':3, 'id':-1})


shape of event list (p_x_y_n_id_t) (6, 147960887)
id min 1
id max 5638399
CPU times: user 15.5 s, sys: 11.2 s, total: 26.7 s
Wall time: 14.4 s


treatment and plotting of data

In [113]:
#binning of each detector panel in t_step bins
panel_0 = da3['panel',0].hist(t=t_step)
panel_1 = da3['panel',1].hist(t=t_step)
panel_2 = da3['panel',2].hist(t=t_step)

p0 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 0].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_0.max().value, aspect='equal',title='panel 0')
p1 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 1].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_0.max().value, aspect='equal',title='panel 1')
p2 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 2].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_0.max().value, aspect='equal',title='panel 2')

#p0.children[0].ax.set_ylim(1280, 0)
#p1.children[0].ax.set_ylim(1280, 0)
#p2.children[0].ax.set_ylim(1280, 0)

p0.children[0].ax.set_xlim(1280, 0)
p1.children[0].ax.set_xlim(1280, 0)
p2.children[0].ax.set_xlim(1280, 0)

l1 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p1.children[1].children[0].children[1], 'value'))
l2 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p2.children[1].children[0].children[1], 'value'))
Box([[p2, p0, p1]])

Box(children=(HBox(children=(Box(children=(InteractiveFig2d(children=(HBar(), HBox(children=(VBar(children=(To…

In [6]:
print(f.filename)
print(f.file)
print(f.driver)
print(f.mode)


/Users/justinbergmann/work_flow/test_data/xe_20_NMX.h5
<HDF5 file "xe_20_NMX.h5" (mode r)>
sec2
r


In [7]:
da.coords['panel'] = id_list // sc.index(2000000)
da2 = da.group('panel').bin(x=pix, y=pix)




up data reagarding ids

In [8]:
print(panel_0.values.shape)
print( "t_min",t_list.min().value, "t_max",t_list.max().value)
t_min =t_list.min().value 
t_max =t_list.max().value

(1638400, 40)
t_min 0.09283190157755548 t_max 0.15202125872662284



ts with slicer to scan TOF

which time of the pulse do we chose as T0 for TOF

In [ ]:
#p1f = da3.fold('id', sizes={'xpix':1280, 'ypix':-1})['panel', 0].hist(t=t_step)
#p11 = da3['panel',0] #.hist(t=t_step)
#p11[0].values

dd 2D binning (x and y)

In [ ]:
#p12 = p11.bins
#sc.show(p1f)

In [ ]:
#p1n = p1f.values
#print(p1n.shape)
#p1n[0][0][0]


In [ ]:
#s1 = p1f['t',0]
#s2 = s1.values
#print(s2.shape,s2[0][0])

#s1

In [ ]:
#p11.values

getting all important things to wrire about the data!

In [9]:
c0 = panel_0.values
c0.shape
t0 = panel_0.coords['t'].values
id0 = panel_0.coords['id'].values
c0

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

3D binning

In [ ]:
tbin=300
hist3=binned.hist(t=tbin)

histrogramm 3D data


plot t slice

In [ ]:
#nb=60
#times=hist3.coords['t']
#print(len(times))
#print(len(times),times[nb],times[nb+1])
#hist3['t',nb].transpose().plot(aspect='equal')

nn with pixel id

In [ ]:
hist3

In [ ]:
# del hist3.coords['tc']
# del hist3.coords['xc']
# del hist3.coords['yc']

In [ ]:
hist4 = hist3.copy()
for name in list(hist4.coords.keys()):
    hist4.coords[f'{name}c'] = sc.midpoints(hist4.coords[name])
hist4

In [ ]:
300*300*200

In [ ]:
hist5 = sc.flatten(hist4, to='row')
ava = np.average(hist5.values) #avarage of values to use as cut off creteria
sig = np.std(hist5.values)
cut = ava + sig*10
#print("background substraction",cut, ava, sig)
select = hist5.data > sc.scalar(cut, unit='counts') # cut of background
filtered = hist5[select]
filtered.coords['tc'] *= 10
filtered

In [ ]:
ava = np.average(hist5.values)
sig = np.std(hist5.values)
#print(ava, sig)

In [ ]:
hist5

In [ ]:
pp.scatter3d(filtered, x='xc', y='yc', z='tc', pixel_size=0.001)#,figsize=(550, 550))

In [ ]:
nbin= 1280**2
print(nbin)
tbin = 12
id_binned = da.bin(id=nbin)

id_hist=id_binned.hist(t=tbin)


id_hist.sum

In [ ]:
#f_out = h5py.File('/Users/justinbergmann/work_flow/test_out/test.h5','w')

detector postion

In [21]:
origen = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['distance'][0].decode()
origen = list(np.float_(origen.split()))
origen

[0.282776, -0.25, 157.697]

In [25]:
origen = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['distance'][...]
ol = origen[0].split()

print = (ol)
det_or = []
for i in range(len(ol)):
    det_or.append(float(ol[i]))
x = float(ol[0])
det_or

[0.282776, -0.25, 157.697]

In [26]:
print( f['entry1/instrument/instrument_xml/data'])

<HDF5 dataset "data": shape (1,), type "|S5382">


In [81]:

def CDist2(A,B):
#calculate distance betweenn two points
        dist = len3dvec(twoP_to_vec(A, B))
        return dist



def len3dvec(vec):
## calculates lengh of a 3D vecor
## input as list
        a = np.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
        return a

def twoP_to_vec(A,B):
#creates vector between two points
        vec = np.array([B[0]-A[0], B[1]-A[1], B[2]-A[2]])

        return vec

In [33]:
xml  = str(f['entry1/instrument/instrument_xml/data'][...][0]).split('\\n')
xml

['b\'<?xml version="1.0" encoding="UTF-8"?>',
 '<!-- IDF generated using McStas McDisplay and the Mantid backend -->',
 '<!-- For help on the notation used to specify an Instrument Definition File see http://www.mantidproject.org/IDF -->',
 '<instrument name="NMX_Mantid.out" valid-from   ="1900-01-31 23:59:59"',
 'valid-to     ="2100-01-31 23:59:59" last-modified="Tue Jan 24 10:03:49 2023">',
 '<defaults>',
 '\\t<length unit="meter"/>',
 '\\t<angle unit="degree"/>',
 '\\t<reference-frame>',
 '\\t\\t<!-- The z-axis is set parallel to and in the direction of the beam. The y-axis points up and the coordinate system is right handed. -->',
 '\\t\\t<along-beam axis="z"/>',
 '\\t\\t<pointing-up axis="y"/>',
 '\\t\\t<handedness val="right"/>',
 '\\t</reference-frame>',
 '\\t\\t<default-view axis-view="z-"/>',
 '\\t</defaults>',
 '',
 '<!-- LIST OF PHYSICAL COMPONENTS (which the instrument consists of) -->',
 '',
 '<type name="Othercomp"></type>',
 '',
 '<component type="sourceMantid-type" name

In [87]:
comp = False
det = False
source = False
sample = False
sample_pos = [0,0,0]
source_pos = [0,0,0]
d_list = []
for i in range(len(xml)):
    ls = xml[i].replace('<t',' ').replace('>',' ').replace('"',' ').replace('<',' ').replace('\\t',' ').split()
    
    if len(ls) >= 1:
        if ls[0] == 'component':
            det = False
            source = False
            sample = False
            comp = True
            if ls[2].split('-')[0] == 'MonNDtype':

                det = True
                d_list.append([int(ls[2].split('-')[1])])
            elif ls[2] == 'sourceMantid-type':
                source = True
            elif ls[2] == 'sampleMantid-type':
                sample = True
            comp = True
     #   if ls[1].split('-')[0] == 'type="MonNDtype':
            print("1",ls)
    if len(ls) >= 1:
        if ls[0] == 'type':
            comp = False
    if len(ls) >= 1:
        if ls[0] == 'location' or ls[0] == 'location':
            print("3",ls)
            if comp == True and det == True:
            #print("1",ls)
            
                print("2",ls)
                xyz = [float(ls[2]),float(ls[4]),float(ls[6])]
                d_list[len(d_list)-1].append(xyz)
            elif comp == True and source == True: 
              source_pos = [float(ls[2]),float(ls[4]),float(ls[6])]
            elif comp == True and sample == True: 
               sample_pos = [float(ls[2]),float(ls[4]),float(ls[6])]
print(d_list)
print(sample_pos,source_pos)
print(CDist2(source_pos, sample_pos))

1 ['component', 'type=', 'sourceMantid-type', 'name=', 'sourceMantid']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'sampleMantid-type', 'name=', 'sampleMantid']
3 ['location', 'x=', '0.53123', 'y=', '0', 'z=', '157.405', 'rot=', '0.303181121242003', 'axis-x=', '0', 'axis-y=', '1', 'axis-z=', '0', '/']
1 ['component', 'type=', 'line-sourceMantid-0']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'line-sourceMantid-1']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'line-sourceMantid-2']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'line-sampleMantid-0']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'line-sampleMantid-1']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'line-sampleMantid-2']
3 ['location', 'x=', '0', 'y=', '0', 'z=', '0', '/']
1 ['component', 'type=', 'MonNDtype-0', 'name=', 'nD_Mant

getting the crystal orientation

In [19]:
print((f['entry1']['simulation']['Param'].keys()))
print((f['entry1']['simulation']['Param']['XtalPhiX']))
phix=float(list(str(f['entry1']['simulation']['Param']['XtalPhiX'][...][0]))[2])
phiy=float(list(str(f['entry1']['simulation']['Param']['XtalPhiY'][...][0]))[2])
phiz=float(list(str(f['entry1']['simulation']['Param']['XtalPhiZ'][...][0]))[2])
#str(phix[0])
#int(list(str(phix[0]))[2])
print(phix,phiy,phiz)
cor=[phix,phiy, phiz]
cryst_or = np.array(cor)
cryst_or

<KeysViewHDF5 ['DetectorDist', 'DetectorX', 'DetectorY', 'L_max', 'L_min', 'LambdaMax', 'LambdaMin', 'Phase2A', 'Phase2B', 'PinholeCollimator_divergence', 'PinholeCollimator_height', 'PinholeCollimator_width', 'Slit1_height', 'Slit1_width', 'Slit2_height', 'Slit2_width', 'TT', 'XtalMosaicity', 'XtalPhiX', 'XtalPhiY', 'XtalPhiZ', 'XtalSize_depth', 'XtalSize_height', 'XtalSize_width', 'frac', 'lmax', 'lmin', 'mon_shift']>
<HDF5 dataset "XtalPhiX": shape (1,), type "|S2">
2.0 0.0 0.0


array([2., 0., 0.])

In [ ]:
#grp = f_out.create_group("raw_data_1")

#grp.__setitem__('beamline','NMX')
#grp.__setitem__('definition','TOFRAW')
#det1 = grp.create_group("detector_1")
#dset = det1.create_dataset("hist_data",data=data1)

#beamline = grp.create_dataset("beamline", ("NMX"))
#print(beamline.name)
#beamline = "NMX"
#"NMX" = grp['beamline'] 

In [10]:
print(filename)

/Users/justinbergmann/work_flow/test_data/xe_20_NMX.h5


In [70]:
no = filename.split('/')
print(no)
name_out= no[-1].split('.')[0]
print(name_out)
file_out = '/Users/justinbergmann/work_flow/test_out/'+name_out+'_out.h5'
print(file_out)

['', 'Users', 'justinbergmann', 'work_flow', 'test_data', 'xe_20_NMX.h5']
xe_20_NMX
/Users/justinbergmann/work_flow/test_out/xe_20_NMX_out.h5


In [89]:
with h5py.File(file_out, 'w') as fo:
## create output nexus file
   fo.attrs['default'] = 'NMX_data'
   nxentry = fo.create_group('NMX_data')
   nxentry.attrs["NX_class"] = 'NXentry'
   nxentry.attrs['default'] = 'data'
   nxentry.__setitem__('beamline','NMX')
   nxentry.__setitem__('definition','TOFRAW')

#SAMPLE
   nx_sample = nxentry.create_group('NXsample')
   nx_sample.__setitem__('name','Single_crystal')
#SOURCE
   nx_inst = nxentry.create_group('instrument')
   nx_source = nxentry.create_group('NXsource')
   nx_source.__setitem__('name','European Spallation Source')
   nx_source.__setitem__('short_name','ESS')
   nx_source.__setitem__('type','Spallation Neutron Source')
   nx_source.__setitem__('distance',-CDist2(source_pos, sample_pos))
   nx_source.__setitem__('probe','neutron')
   nx_source.__setitem__('target_material','W')


   for i in range(n_det):
      panel = da3['panel',i].hist(t=t_step)
      p_name = "panel_"+str(i)
      nxdata = nxentry.create_group(p_name) 
      nxdata.attrs["NX_class"] = 'NXdata'
      nxdata.attrs['signal'] = 'counts' 
      #ad counds
      #counts = nxdata.create_dataset('counts', data=panel_0.values)
      counts = nxdata.create_dataset('counts', data=panel.values)


      counts.attrs['units'] = 'counts'
      counts.attrs['long_name'] = 'counts of panel'+str(i)+' (counts)'
      
      id_bin = nxdata.create_dataset('pix_id', data=panel.coords['id'].values)
      id_bin.attrs['units'] = ''
      id_bin.attrs['long_name'] = 'pixel ID'
      
      t_bin = nxdata.create_dataset('t_bin', data=panel.coords['t'].values)
      t_bin.attrs['units'] = 'ms'
      t_bin.attrs['long_name'] = 't_bin TOF (ms)'

    
#   nxdata = nxentry.create_group('panel_0') 
#   nxdata.attrs["NX_class"] = 'NXdata'
#   nxdata.attrs['signal'] = 'counts' 
   #ad counds
#   counts = nxdata.create_dataset('counts', data=panel_0.values)
#   counts.attrs['units'] = 'counts'
#   counts.attrs['long_name'] = 'counts of panel'+str(i)+' (counts)'
      
 #  id_bin = nxdata.create_dataset('id_bin', data=panel_0.coords['id'].values)
 #  id_bin.attrs['units'] = ''
 #  id_bin.attrs['long_name'] = 'pixel ID'
      
 #  t_bin = nxdata.create_dataset('t_bin', data=panel_0.coords['t'].values)
 #  t_bin.attrs['units'] = 'ms'
 #  t_bin.attrs['long_name'] = 't_bin TOF (ms)'

 


    #nxdata = nxentry.create_group('data')
    #nxdata.attrs["NX_class"] = 'NXdata'
    #nxdata.attrs['signal'] = 'counts'
    #nxdata.attrs['axes'] = 'two_theta'
    #nxdata.attrs['two_theta_indices'] = [0,]
 
  #  tth = nxdata.create_dataset('data1', data=data1)
   # tth.attrs['units'] = 'counts'
    #tth.attrs['long_name'] = 'data1 (counts)'

#    id_bin = nxdata.create_dataset('id_bin', data=id_bin)
#    id_bin.attrs['units'] = ''
#    id_bin.attrs['long_name'] = 'pixel ID'
    
 #   t_bin = nxdata.create_dataset('t_bin', data=tl_bin)
 #   t_bin.attrs['units'] = 'ms'
 #   t_bin.attrs['long_name'] = 't_bin TOF (ms)'

#    counts = nxdata.create_dataset('counts', data=data2)
#    counts.attrs['units'] = 'counts'
#    counts.attrs['long_name'] = 'counts (bined counts)'





#    c_or = nxinst.create_dataset('crystal_orientation',data=cryst_or)
#    c_or.attrs['unit'] = 'degrees'
#    c_or.attrs['long_name'] = 'crystal orientation in Phi (degrees)'
    

   fo.close()


In [ ]:
#print(grp['beamline'].items())
fo.close()

In [ ]:

#f_out.close()

In [ ]:
f.close()  
del id_list
del x_list
del y_list
del t_list
del id_hist
del hist3

In [ ]:
current, peak = tracemalloc.get_traced_memory()
print('2 current memory [MB]: {}, peak memory [MB]: {} '.format(round((current/(1024*1024)), 2), round((peak /(1024*1024) ), 2) ))
# stopping the library
tracemalloc.stop()
print("neded time (h:mm:ss): ", datetime.now() - startTime)